In [331]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [332]:
import pandas as pd

In [333]:
X_train=pd.read_csv("../DATA/UCI_HAR_Dataset/train/X_train.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])
y_train=pd.read_csv("../DATA/UCI_HAR_Dataset/train/y_train.txt", delim_whitespace=True, names=["label"])
y_train['label']=y_train['label']-1

X_test=pd.read_csv("../DATA/UCI_HAR_Dataset/test/X_test.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])
y_test=pd.read_csv("../DATA/UCI_HAR_Dataset/test/y_test.txt", delim_whitespace=True, names=["label"])
y_test['label']=y_test['label']-1

In [334]:
trainDF = pd.concat([X_train,y_train],axis=1) 

testDF=pd.concat([X_test, y_test],axis=1)

In [335]:
testDF.values

array([[ 0.25717778, -0.02328523, -0.01465376, ...,  0.27680104,
        -0.0579783 ,  4.        ],
       [ 0.28602671, -0.01316336, -0.11908252, ...,  0.28134292,
        -0.08389801,  4.        ],
       [ 0.27548482, -0.02605042, -0.11815167, ...,  0.28008303,
        -0.0793462 ,  4.        ],
       ...,
       [ 0.34996609,  0.03007744, -0.11578796, ...,  0.27447878,
         0.18118355,  1.        ],
       [ 0.23759383,  0.01846687, -0.09649893, ...,  0.26478161,
         0.18756291,  1.        ],
       [ 0.15362719, -0.01843651, -0.13701846, ...,  0.26393619,
         0.1881034 ,  1.        ]])

In [347]:
training_data = torch.utils.data.TensorDataset(torch.tensor(X_train.values).float(), torch.as_tensor(y_train.values).squeeze())
test_data=torch.utils.data.TensorDataset(torch.tensor(X_test.values).float(), torch.as_tensor(y_test.values).squeeze())

In [337]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#for X, y in test_dataloader:
#    print("Shape of X [N, C, H, W]: ", X.shape)
#    print("Shape of y: ", y.shape, y.dtype)
#    break

In [338]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(561, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6)
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=561, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=6, bias=True)
  )
)


In [339]:
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=561, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=6, bias=True)
  )
)

In [340]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [341]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        #print("pred:",pred.shape, pred.dtype," y:" ,y.shape,y.dtype)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [342]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [343]:
epochs = 55
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.826339  [    0/ 7352]
loss: 1.814025  [ 6400/ 7352]
Test Error: 
 Accuracy: 23.6%, Avg loss: 1.751781 

Epoch 2
-------------------------------
loss: 1.694742  [    0/ 7352]
loss: 1.811628  [ 6400/ 7352]
Test Error: 
 Accuracy: 18.5%, Avg loss: 1.710078 

Epoch 3
-------------------------------
loss: 1.601068  [    0/ 7352]
loss: 1.801475  [ 6400/ 7352]
Test Error: 
 Accuracy: 21.5%, Avg loss: 1.670729 

Epoch 4
-------------------------------
loss: 1.524680  [    0/ 7352]
loss: 1.781127  [ 6400/ 7352]
Test Error: 
 Accuracy: 31.1%, Avg loss: 1.629987 

Epoch 5
-------------------------------
loss: 1.459574  [    0/ 7352]
loss: 1.748627  [ 6400/ 7352]
Test Error: 
 Accuracy: 43.3%, Avg loss: 1.585618 

Epoch 6
-------------------------------
loss: 1.402464  [    0/ 7352]
loss: 1.703891  [ 6400/ 7352]
Test Error: 
 Accuracy: 51.9%, Avg loss: 1.536487 

Epoch 7
-------------------------------
loss: 1.351626  [    0/ 7352]
loss: 1.648775  [ 

In [344]:
torch.save(model.state_dict(), "activityModel.pth")

In [345]:
# Let's do some predictions

with torch.no_grad():
    for X,y in test_dataloader:
        pred = model(X)
        print("Probabilities:",pred)
        print("Predicted class:",torch.argmax(pred, dim=1))
        print("match:",torch.argmax(pred, dim=1)==y)

Probabilities: tensor([[-0.4578, -0.2504, -4.5227,  2.4778,  3.0469, -0.1906],
        [-1.0766, -1.0802, -5.8128,  3.6429,  3.9519,  0.4558],
        [-1.3241, -1.5037, -6.0579,  4.0162,  4.3158,  0.5461],
        [-1.4353, -1.5039, -6.1261,  4.1091,  4.5459,  0.4366],
        [-1.4365, -1.3462, -5.9890,  3.9830,  4.5405,  0.2635],
        [-1.4231, -1.3281, -5.9692,  3.9979,  4.5186,  0.2327],
        [-1.4941, -1.4766, -6.2252,  4.2107,  4.6081,  0.3971],
        [-1.7264, -1.6938, -6.3945,  4.4101,  4.6295,  0.7771],
        [-1.9018, -1.8088, -6.6163,  4.6123,  4.8055,  0.8927],
        [-1.8066, -2.0537, -6.7479,  4.8401,  4.8052,  0.9767],
        [-1.7828, -1.9591, -6.6883,  4.7907,  4.8256,  0.8345],
        [-1.7711, -2.2211, -6.8582,  4.8240,  4.6692,  1.3672],
        [-1.7829, -2.1750, -6.7591,  4.7868,  4.7334,  1.2241],
        [-1.6467, -1.8854, -6.6046,  4.5346,  4.7899,  0.8239],
        [-1.7324, -1.7544, -6.6457,  4.5063,  4.6474,  1.0182],
        [-1.0213, -1.2663

In [346]:
!pwd

/home/jupyter/hlg-mos input privacy
